<a href="https://colab.research.google.com/github/nazaninaslani/TimeSeriesAnalysis-PredictingModel/blob/master/CleanningTimeSeriesCOVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [118]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [119]:
#time_series_covid19_recovered_global
downloaded = drive.CreateFile({'id':'1Bnatcc4whwprOVMGgrvHvnf4x_o9TIEE'}) 
downloaded.GetContentFile('time_series_covid19_recovered_global.csv')

In [120]:
#time_series_covid19_deaths_global
downloaded = drive.CreateFile({'id':'1Ltzi51t5Vg5DelP20KiuiwqLIZ3gKPZp'}) 
downloaded.GetContentFile('time_series_covid19_deaths_global.csv')

In [121]:
#time_series_covid19_deaths_US
downloaded = drive.CreateFile({'id':'1An479yRn6d4SgGjkfE6rYqe0OYXUDUxJ'})
downloaded.GetContentFile('time_series_covid19_deaths_US.csv')

In [122]:
#time_series_covid19_confirmed_global
downloaded = drive.CreateFile({'id':'1SZxipnNzqisnmSjwnwf34ZABq_uxw6iT'}) 
downloaded.GetContentFile('time_series_covid19_confirmed_global.csv')

In [123]:
#time_series_covid19_confirmed_US
downloaded = drive.CreateFile({'id':'1M-5PhDNIYguXSA-Gzl0uMSdCPZpx7lN5'}) 
downloaded.GetContentFile('time_series_covid19_confirmed_US.csv')

In [124]:
import os

In [125]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime
import random
%matplotlib inline

In [126]:
#to change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'(:f)'.format})

In [127]:
#increase the size of sns plots
sns.set(rc={'figure.figsize':(12,10)})

In [128]:
#import sys
# see all the rows/columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [129]:
#loading the cumulative raw data
raw_data_confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
raw_data_deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
raw_data_Recovered = pd.read_csv('time_series_covid19_recovered_global.csv')
raw_data_confirm_US = pd.read_csv('time_series_covid19_confirmed_US.csv')
raw_data_deaths_US = pd.read_csv('time_series_covid19_deaths_US.csv')

In [130]:
print("The shape of confirmed is:", raw_data_confirmed.shape)
print("The shape of Death is:", raw_data_deaths.shape)
print("The shape of Recovered is:", raw_data_Recovered.shape)
print("The shape of Confirmed US is:", raw_data_confirm_US.shape)
print("The shape of Death US is:", raw_data_deaths_US.shape)

The shape of confirmed is: (266, 163)
The shape of Death is: (266, 163)
The shape of Recovered is: (253, 163)
The shape of Confirmed US is: (3261, 170)
The shape of Death US is: (3261, 171)


In [131]:
raw_data_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174,237,273,281,299,349,367,423,444,484,521,555,607,665,714,784,840,906,933,996,1026,1092,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224,3392,3563,3778,4033,4402,4687,4963,5226,5639,6053,6402,6664,7072,7653,8145,8676,9216,9998,10582,11173,11831,12456,13036,13659,14525,15205,15750,16509,17267,18054,18969,19551,20342,20917,21459,22142,22890,23546,24102,24766,25527,26310,26874,27532,27878,28424,28833,29157,29481,29640,30175,30451,30616,30967
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409,416,433,446,467,475,494,518,539,548,562,584,609,634,663,678,712,726,736,750,766,773,782,789,795,803,820,832,842,850,856,868,872,876,880,898,916,933,946,948,949,964,969,981,989,998,1004,1029,1050,1076,1099,1122,1137,1143,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385,1416,1464,1521,1590,1672,1722,1788,1838,1891,1962,1995,2047,2114,2192,2269,2330,2402
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648,4838,4997,5182,5369,5558,5723,5891,6067,6253,6442,6629,6821,7019,7201,7377,7542,7728,7918,8113,8306,8503,8697,8857,8997,9134,9267,9394,9513,9626,9733,9831,9935,10050,10154,10265,10382,10484,10589,10698,10810,10919,11031,11147,11268,11385,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370,376,390,428,439,466,501,525,545,564,583,601,601,638,646,659,673,673,696,704,713,717,717,723,723,731,738,738,743,743,743,745,745,747,748,750,751,751,752,752,754,755,755,758,760,761,761,761,761,761,761,762,762,762,762,762,763,763,763,763,764,764,764,765,844,851,852,852,852,852,852,852,852,852,853,853,853,853,854,854,855,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7,7,8,8,8,10,14,16,17,19,19,19,19,19,19,19,19,19,19,24,24,24,24,25,25,25,25,26,27,27,27,27,3

In [132]:
#creating a Dates list
raw_data_confirm_US.columns

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_',
       ...
       '6/19/20', '6/20/20', '6/21/20', '6/22/20', '6/23/20', '6/24/20', '6/25/20', '6/26/20', '6/27/20', '6/28/20'], dtype='object', length=170)

In [133]:
#creating a Dates list
column_dates = raw_data_confirm_US.columns
column_dates = column_dates.drop(labels=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_'])

In [134]:
#Droping columns we do not need in US dfs
raw_data_confirm_US.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2'], axis=1, inplace = True)
raw_data_deaths_US.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2'], axis=1, inplace = True)

In [135]:
#Averaging the lat & lang
lat_lon_Avg = raw_data_confirm_US.groupby(['Province_State', 'Country_Region'],as_index = False).agg({'Lat':'median','Long_':'median'})

In [136]:
# summing the values
raw_data_confirm_US = raw_data_confirm_US.groupby(['Province_State', 'Country_Region'],as_index=False)[column_dates].sum()
raw_data_deaths_US = raw_data_deaths_US.groupby(['Province_State', 'Country_Region'],as_index=False)[column_dates].sum()

In [137]:
#left joining the AVG lat & lan
raw_data_confirm_US = raw_data_deaths_US.merge(lat_lon_Avg, how='left', left_on = ['Province_State', 'Country_Region'], right_on=['Province_State', 'Country_Region'])
raw_data_deaths_US = raw_data_deaths_US.merge(lat_lon_Avg, how='left', left_on = ['Province_State', 'Country_Region'], right_on=['Province_State', 'Country_Region'])

In [138]:
raw_data_confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '6/19/20', '6/20/20', '6/21/20', '6/22/20', '6/23/20', '6/24/20', '6/25/20', '6/26/20', '6/27/20', '6/28/20'], dtype='object', length=163)

In [139]:
raw_data_confirm_US.columns

Index(['Province_State', 'Country_Region', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       ...
       '6/21/20', '6/22/20', '6/23/20', '6/24/20', '6/25/20', '6/26/20', '6/27/20', '6/28/20', 'Lat', 'Long_'], dtype='object', length=163)

In [140]:
#Rename the columns
raw_data_confirm_US = raw_data_confirm_US.rename(columns={'Province_State': 'Province/State', 'Country_Region':'Country/Region','Long_':'Long'})
raw_data_deaths_US = raw_data_deaths_US.rename(columns={'Province_State': 'Province/State', 'Country_Region':'Country/Region','Long_':'Long'})

In [141]:
raw_data_confirm_US.columns

Index(['Province/State', 'Country/Region', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       ...
       '6/21/20', '6/22/20', '6/23/20', '6/24/20', '6/25/20', '6/26/20', '6/27/20', '6/28/20', 'Lat', 'Long'], dtype='object', length=163)

In [142]:
#Re-ordering the columns
raw_data_confirm_US = raw_data_confirm_US[raw_data_confirmed.columns]
raw_data_deaths_US = raw_data_deaths_US[raw_data_confirmed.columns]

In [143]:
print("The shape of confirmed is:",raw_data_confirmed.shape)
print("The shape of confirmed US is:",raw_data_confirm_US.shape)
print()
print("The shape of Death is:",raw_data_deaths.shape)
print("The shape of Death US is:",raw_data_deaths_US.shape)

The shape of confirmed is: (266, 163)
The shape of confirmed US is: (58, 163)

The shape of Death is: (266, 163)
The shape of Death US is: (58, 163)


In [144]:
#Filtering out the us
raw_data_confirmed = raw_data_confirmed[raw_data_confirmed['Country/Region']!='US']
raw_data_deaths = raw_data_deaths[raw_data_deaths['Country/Region']!='US']

In [145]:
#Union the 2-dfs
raw_data_confirmed = pd.concat([raw_data_confirmed, raw_data_confirm_US], ignore_index=True)
raw_data_deaths = pd.concat([raw_data_deaths, raw_data_deaths_US], ignore_index = True)

In [146]:
# un-pivoting the data
raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region','Lat', 'Long'], var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region','Lat', 'Long'], var_name=['Date'])
raw_data_Recovered2 = pd.melt(raw_data_Recovered, id_vars=['Province/State', 'Country/Region','Lat', 'Long'], var_name=['Date'])


In [147]:
print("The shape of confirmed2 is:", raw_data_confirmed2.shape)
print("The shape of Death2 is:", raw_data_deaths2.shape)
print("The shape of Recovered2 is:", raw_data_Recovered2.shape)

The shape of confirmed2 is: (51357, 6)
The shape of Death2 is: (51357, 6)
The shape of Recovered2 is: (40227, 6)


In [148]:
#Converting the new column to dates
raw_data_confirmed2['Date']=pd.to_datetime(raw_data_confirmed2['Date'])
raw_data_deaths2['Date']=pd.to_datetime(raw_data_deaths2['Date'])
raw_data_Recovered2['Date']=pd.to_datetime(raw_data_Recovered2['Date'])

In [149]:
raw_data_deaths2.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', 'Date', 'value'], dtype='object')

In [150]:
#Renewing the values
raw_data_confirmed2.columns=raw_data_confirmed2.columns.str.replace('value','confirmed')
raw_data_deaths2.columns=raw_data_deaths2.columns.str.replace('value','Death')
raw_data_Recovered2.columns=raw_data_Recovered2.columns.str.replace('value','Recovered')


In [151]:
#Investigating the Null values
raw_data_Recovered2.isnull().sum()

Province/State    29574
Country/Region        0
Lat                   0
Long                  0
Date                  0
Recovered             0
dtype: int64

In [152]:
#Dealing with Null values
raw_data_confirmed2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)
raw_data_deaths2['Province/State'].fillna(raw_data_deaths2['Country/Region'], inplace=True)
raw_data_Recovered2['Province/State'].fillna(raw_data_Recovered2['Country/Region'], inplace=True)

raw_data_confirmed2.isnull().sum()

Province/State    0
Country/Region    0
Lat               0
Long              0
Date              0
confirmed         0
dtype: int64

In [153]:
print("The shape of confirmed2 is:", raw_data_confirmed2.shape)
print("The shape of Death2 is:", raw_data_deaths2.shape)
print("The shape of Recovered2 is:", raw_data_Recovered2.shape)

The shape of confirmed2 is: (51357, 6)
The shape of Death2 is: (51357, 6)
The shape of Recovered2 is: (40227, 6)


In [154]:
raw_data_Recovered2.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', 'Date', 'Recovered'], dtype='object')

In [155]:
#left join

#confirmed with death
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State', 'Country/Region', 'Date', 'Death']],
                                      how = 'left',
                                      left_on=['Province/State', 'Country/Region', 'Date'],
                                      right_on=['Province/State', 'Country/Region', 'Date'])

print('shape of first join',full_join.shape)

shape of first join (51357, 7)


In [156]:
full_join = full_join.merge(raw_data_Recovered2[['Province/State', 'Country/Region', 'Date', 'Recovered']],
                                      how = 'left',
                                      left_on=['Province/State', 'Country/Region', 'Date'],
                                      right_on=['Province/State', 'Country/Region', 'Date'])

print('shape of first join',full_join.shape)

shape of first join (51357, 8)


In [157]:
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Death,Recovered
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0.0
1,Albania,Albania,41.1533,20.1683,2020-01-22,0,0,0.0
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0,0,0.0
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0,0,0.0
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0,0,0.0


In [158]:
#checking for null values (especialy long and lat)
full_join.isnull().sum()

Province/State        0
Country/Region        0
Lat                   0
Long                  0
Date                  0
confirmed             0
Death                 0
Recovered         11448
dtype: int64

In [159]:
#Adding month and year as a new column
full_join['Month-Year'] = full_join['Date'].dt.strftime("%b-%Y")

In [115]:
full_join.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Death,Recovered,Month-Year
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0.0,Jan-2020
1,Albania,Albania,41.1533,20.1683,2020-01-22,0,0,0.0,Jan-2020
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0,0,0.0,Jan-2020
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0,0,0.0,Jan-2020
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0,0,0.0,Jan-2020


In [40]:
#test = full_join[full_join['Province/State']=='Anhui']

#full_join2 = test.copy()

In [41]:
#full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
#full_join2.rename(columns={'confirmed':'confirmed-1', 'Death':'Death-1', 'Recovered':'Recovered-1','Date':'Date Minus 1'}, inplace=True)

In [42]:
#full_join2.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', 'Date Minus 1', 'confirmed-1', 'Death-1', 'Recovered-1', 'Month-Year', 'Date - 1'], dtype='object')

In [43]:
#full_join3 = test.merge(full_join2[['Province/State', 'Country/Region','confirmed-1', 'Death-1', 'Recovered-1','Date - 1','Date Minus 1']],
                        how = 'outer',
                        left_on=['Province/State', 'Country/Region', 'Date'],
                        right_on=['Province/State', 'Country/Region', 'Date - 1'])

In [44]:
#full_join3['confirmed Daily'] = full_join3['confirmed'] - full_join3['confirmed-1']

In [45]:
#test.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Death,Recovered,Month-Year
49,Anhui,China,31.8257,117.2264,2020-01-22,1,0,0.0,Jan-2020
372,Anhui,China,31.8257,117.2264,2020-01-23,9,0,0.0,Jan-2020
695,Anhui,China,31.8257,117.2264,2020-01-24,15,0,0.0,Jan-2020
1018,Anhui,China,31.8257,117.2264,2020-01-25,39,0,0.0,Jan-2020
1341,Anhui,China,31.8257,117.2264,2020-01-26,60,0,0.0,Jan-2020


In [46]:
#full_join2.head()

,Province/State,Country/Region,Lat,Long,Date Minus 1,confirmed-1,Death-1,Recovered-1,Month-Year,Date - 1
49,Anhui,China,31.8257,117.2264,2020-01-22,1,0,0.0,Jan-2020,2020-01-23
372,Anhui,China,31.8257,117.2264,2020-01-23,9,0,0.0,Jan-2020,2020-01-24
695,Anhui,China,31.8257,117.2264,2020-01-24,15,0,0.0,Jan-2020,2020-01-25
1018,Anhui,China,31.8257,117.2264,2020-01-25,39,0,0.0,Jan-2020,2020-01-26
1341,Anhui,China,31.8257,117.2264,2020-01-26,60,0,0.0,Jan-2020,2020-01-27


In [47]:
#full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Death,Recovered,Month-Year,confirmed-1,Death-1,Recovered-1,Date - 1,Date Minus 1,confirmed Daily
0,Anhui,China,31.8257,117.2264,2020-01-22,1.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN
1,Anhui,China,31.8257,117.2264,2020-01-23,9.0,0.0,0.0,Jan-2020,1.0,0.0,0.0,2020-01-23,2020-01-22,8.0
2,Anhui,China,31.8257,117.2264,2020-01-24,15.0,0.0,0.0,Jan-2020,9.0,0.0,0.0,2020-01-24,2020-01-23,6.0
3,Anhui,China,31.8257,117.2264,2020-01-25,39.0,0.0,0.0,Jan-2020,15.0,0.0,0.0,2020-01-25,2020-01-24,24.0
4,Anhui,China,31.8257,117.2264,2020-01-26,60.0,0.0,0.0,Jan-2020,39.0,0.0,0.0,2020-01-26,2020-01-25,21.0


In [160]:
################### Breaking the numbers by Day ##################
# Applying it on all dataset
full_join2 = full_join.copy()
#creating a new df
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'confirmed':'confirmed-1', 'Death':'Death-1', 'Recovered':'Recovered-1','Date':'Date Minus 1'}, inplace=True)

In [161]:
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','confirmed-1', 'Death-1', 'Recovered-1','Date - 1','Date Minus 1']],
                        how = 'outer',
                        left_on=['Province/State', 'Country/Region', 'Date'],
                        right_on=['Province/State', 'Country/Region', 'Date - 1'])

In [162]:
full_join3.head(10)

,Province/State,Country/Region,Lat,Long,Date,confirmed,Death,Recovered,Month-Year,confirmed-1,Death-1,Recovered-1,Date - 1,Date Minus 1
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
1,Albania,Albania,41.1533,20.1683,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
5,Antigua and Barbuda,Antigua and Barbuda,17.0608,-61.7964,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
6,Argentina,Argentina,-38.4161,-63.6167,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
7,Armenia,Armenia,40.0691,45.0382,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
8,Australian Capital Territory,Australia,-35.4735,149.0124,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT
9,New South Wales,Australia,-33.8688,151.2093,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT


In [163]:
full_join3['Confirmed Daily'] = full_join3['confirmed'] - full_join3['confirmed-1']
full_join3['Death Daily'] = full_join3['Death'] - full_join3['Death-1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered-1']

print(full_join3.shape)

(51680, 17)


In [164]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Death,Recovered,Month-Year,confirmed-1,Death-1,Recovered-1,Date - 1,Date Minus 1,Confirmed Daily,Death Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
1,Albania,Albania,41.1533,20.1683,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0.0,0.0,0.0,Jan-2020,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN


In [165]:
#Adding manually the numbers for first day
full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['confirmed']
full_join3['Death Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Death']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [166]:
#deleting columns
del full_join3['confirmed-1']
del full_join3['Death-1']
del full_join3['Recovered-1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']

In [167]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed,Death,Recovered,Month-Year,Confirmed Daily,Death Daily,Recovered Daily
0,Afghanistan,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
1,Albania,Albania,41.1533,20.1683,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
2,Algeria,Algeria,28.0339,1.6596,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
3,Andorra,Andorra,42.5063,1.5218,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0
4,Angola,Angola,-11.2027,17.8739,2020-01-22,0.0,0.0,0.0,Jan-2020,0.0,0.0,0.0


In [ ]:
#Creating additional slicer for easy of use
